In [1]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/summarization/dailymail/translated-dailymail-train.json
# !wget https://f000.backblazeb2.com/file/malay-dataset/summarization/cnn-news/translated-cnn-train.json

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'mesolitica-tpu.json'

In [3]:
import numpy as np
from numpy.random import default_rng
import random
import collections
import re
import tensorflow as tf
from tqdm import tqdm
import tokenization
import json
from google.cloud import storage
import mp

max_seq_length_encoder = 512
max_seq_length_decoder = 512

EOS_ID = 1

tokenizer = tokenization.FullTokenizer(
    vocab_file='pegasus.wordpiece', do_lower_case=False
)

In [4]:
def create_int_feature(values):
    feature = tf.train.Feature(
        int64_list=tf.train.Int64List(value=list(values))
    )
    return feature

def write_instance_to_example_file(X, Y, output_file):
    writer = tf.python_io.TFRecordWriter(output_file)
    for i in range(len(X)):
        input_ids = X[i]
        target_ids = Y[i]
        while len(input_ids) < max_seq_length_encoder:
            input_ids.append(0)
        while len(target_ids) < max_seq_length_decoder:
            target_ids.append(0)

        features = collections.OrderedDict()
        features['input_ids'] = create_int_feature(input_ids)
        features['target_ids'] = create_int_feature(target_ids)
        tf_example = tf.train.Example(
            features=tf.train.Features(feature=features)
        )
        writer.write(tf_example.SerializeToString())

    tf.logging.info('Wrote %d total instances', inst_index)

In [5]:
def get_feature(x, y):
    tokens = tokenizer.tokenize(x)
    if len(tokens) > (max_seq_length_encoder - 2):
        tokens = tokens[: max_seq_length_encoder - 2]
    tokens = tokens

    tokens_y = []
    for y_ in y:
        tokens_y.extend(tokenizer.tokenize(y_))
    if len(tokens_y) > (max_seq_length_decoder - 1):
        tokens_y = tokens_y[: max_seq_length_decoder - 1]
    
    tokens_y = tokenizer.convert_tokens_to_ids(tokens_y)
    tokens_y = tokens_y + [EOS_ID]
    return tokenizer.convert_tokens_to_ids(tokens), tokens_y

In [6]:
def write_instance_to_example_file(X, Y, output_file):
    writer = tf.python_io.TFRecordWriter(output_file)
    for i in range(len(X)):
        input_ids = X[i]
        target_ids = Y[i]
        while len(input_ids) < max_seq_length_encoder:
            input_ids.append(0)
        while len(target_ids) < max_seq_length_decoder:
            target_ids.append(0)

        features = collections.OrderedDict()
        features['input_ids'] = create_int_feature(input_ids)
        features['target_ids'] = create_int_feature(target_ids)
        tf_example = tf.train.Example(
            features=tf.train.Features(feature=features)
        )
        writer.write(tf_example.SerializeToString())

    tf.logging.info('Wrote %d total instances', len(X))

In [7]:
def loop(rows):
    client = storage.Client()
    bucket = client.bucket('mesolitica-tpu-general')
    rows, index = rows
    
    X, Y = [], []
    output_file = f'pegasus-summarization-{index}.tfrecord'
    for i in tqdm(range(len(rows))):
        tokens, tokens_y = get_feature(' '.join(data[i]['ms_article']), [' '.join(data[i]['ms_abstract'])])
        X.append(tokens)
        Y.append(tokens_y)
            
    write_instance_to_example_file(X, Y, output_file)
    blob = bucket.blob(f'pegasus-summarization-data/{output_file}')
    blob.upload_from_filename(output_file)
    os.system(f'rm {output_file}')

In [9]:
with open('translated-cnn-train.json') as fopen:
    data = json.load(fopen)
    
with open('translated-cnn-train.json') as fopen:
    data.extend(json.load(fopen))

In [11]:
import random

random.shuffle(data)

In [13]:
mp.multiprocessing(data, loop, cores = 20)

 95%|█████████▌| 8631/9057 [02:28<00:08, 51.81it/s]

INFO:tensorflow:Wrote 9057 total instances


 96%|█████████▌| 8661/9057 [02:28<00:09, 43.56it/s]

INFO:tensorflow:Wrote 9057 total instances


 96%|█████████▌| 8717/9057 [02:32<00:05, 60.31it/s]

...

INFO:tensorflow:Wrote 9057 total instances


100%|█████████▉| 9026/9057 [02:34<00:00, 78.19it/s]

INFO:tensorflow:Wrote 9057 total instances


100%|██████████| 9057/9057 [02:34<00:00, 58.56it/s]


INFO:tensorflow:Wrote 9057 total instances
INFO:tensorflow:Wrote 9057 total instances
INFO:tensorflow:Wrote 9057 total instances
INFO:tensorflow:Wrote 9057 total instances
INFO:tensorflow:Wrote 9057 total instances
INFO:tensorflow:Wrote 9057 total instances
INFO:tensorflow:Wrote 9057 total instances
INFO:tensorflow:Wrote 9057 total instances
INFO:tensorflow:Wrote 9057 total instances


In [15]:
def _decode_record(record, name_to_features):
    """Decodes a record to a TensorFlow example."""
    example = tf.parse_single_example(record, name_to_features)

    # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
    # So cast all int64 to int32.
    for name in list(example.keys()):
        t = example[name]
        if t.dtype == tf.int64:
            t = tf.to_int32(t)
        example[name] = t

    return example

def input_fn_builder(
    input_files,
    max_seq_length_encoder,
    max_seq_length_decoder,
    max_predictions_per_seq,
    is_training,
    num_cpu_threads = 4,
):
    def input_fn(params):
        batch_size = params['batch_size']

        name_to_features = {
            'input_ids': tf.FixedLenFeature([max_seq_length_encoder], tf.int64),
            'target_ids': tf.FixedLenFeature(
                [max_seq_length_decoder], tf.int64
            ),
        }
        if is_training:
            d = tf.data.Dataset.from_tensor_slices(tf.constant(input_files))
            d = d.repeat()
            d = d.shuffle(buffer_size = len(input_files))
            cycle_length = min(num_cpu_threads, len(input_files))
            d = d.apply(
                tf.contrib.data.parallel_interleave(
                    tf.data.TFRecordDataset,
                    sloppy = is_training,
                    cycle_length = cycle_length,
                )
            )
            d = d.shuffle(buffer_size = 100)
        else:
            d = tf.data.TFRecordDataset(input_files)
            d = d.repeat()
        d = d.apply(
            tf.contrib.data.map_and_batch(
                lambda record: _decode_record(record, name_to_features),
                batch_size = batch_size,
                num_parallel_batches = num_cpu_threads,
                drop_remainder = True,
            )
        )
        return d

    return input_fn

In [16]:
files = tf.io.gfile.glob('gs://mesolitica-tpu-general/pegasus-summarization-data/*.tfrecord')
input_fn = input_fn_builder(files, 512, 512, 0, True)
dataset = input_fn({'batch_size': 1})
dataset = dataset._make_one_shot_iterator().get_next()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use `tf.data.experimental.parallel_interleave(...)`.
Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.
Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.
Instructions for updating:
Use `tf.data.Dataset.map(map_func, num_parallel_calls)` followed by `tf.data.Dataset.batch(batch_size, drop_remainder)`. Static tf.data optimizations will take care of using the fused implementation.

Instructions for

In [17]:
sess = tf.Session()

In [18]:
r = sess.run(dataset)

In [19]:
import tokenization

tokenizer = tokenization.FullTokenizer(
    vocab_file='pegasus.wordpiece', do_lower_case=False
)

In [20]:
tokenizer.convert_ids_to_tokens(r['input_ids'][0])

['(',
 'CN',
 '##N',
 ')',
 '-',
 'Real',
 'Madrid',
 'telah',
 'melakukan',
 'sedikit',
 'sebanyak',
 ',',
 'jadi',
 'sekarang',
 'terser',
 '##ah',
 'kepada',
 'pesaing',
 'hebat',
 'Barcelona',
 'untuk',
 'memainkan',
 'peranan',
 'mereka',
 'jika',
 'final',
 'Cop',
 '##a',
 'del',
 'Re',
 '##y',
 'tahun',
 'ini',
 'akan',
 'menjadi',
 'lanjutan',
 'dari',
 "'",
 'El',
 'Cla',
 '##si',
 '##co',
 "'",
 '.',
 'Mend',
 '##ahului',
 '3',
 '-',
 '0',
 'menentang',
 'Atlet',
 '##ico',
 'Madrid',
 ',',
 'pesaing',
 'utama',
 'Real',
 'yang',
 'lain',
 ',',
 'Cris',
 '##tian',
 '##o',
 'Ronaldo',
 'menjaringkan',
 'dua',
 'penalti',
 'awal',
 'untuk',
 'merangkul',
 'kemenangan',
 'agregat',
 '5',
 '-',
 '0',
 'yang',
 'paling',
 'selesa',
 'di',
 'separuh',
 'akhir',
 '.',
 'Peny',
 '##erang',
 'Portugal',
 'itu',
 'yang',
 'kini',
 'menjalani',
 'larangan',
 'liga',
 'tiga',
 'perlawanan',
 'selepas',
 'kad',
 'merah',
 'kerana',
 'melakukan',
 'kekerasan',
 ',',
 'dis',
 '##ambar',
 'pe

In [21]:
tokenizer.convert_ids_to_tokens(r['target_ids'][0])

['Real',
 'Madrid',
 'menewaskan',
 'pesaing',
 'bandar',
 ',',
 'Atlet',
 '##ico',
 '5',
 '-',
 '0',
 'menerusi',
 'agregat',
 'untuk',
 'mara',
 'ke',
 'final',
 'Cop',
 '##a',
 'del',
 'Re',
 '##y',
 '.',
 'Barcelona',
 'boleh',
 'menyertai',
 'mereka',
 'jika',
 'melindungi',
 'kedudukan',
 '2',
 '-',
 '0',
 'di',
 'Real',
 'Soci',
 '##eda',
 '##d',
 'pada',
 'Rabu',
 '.',
 'Pemimpin',
 'Liga',
 'Perdana',
 'Inggeris',
 ',',
 'Chelsea',
 ',',
 'menjatuhkan',
 'mata',
 'penting',
 'di',
 'West',
 'Brom',
 '.',
 'Fi',
 '##ore',
 '##n',
 '##tina',
 'mencapai',
 'perlawanan',
 'akhir',
 'Piala',
 'Itali',
 'dengan',
 'kemenangan',
 'kuku',
 'ke',
 'atas',
 'U',
 '##din',
 '##ese',
 '.',
 '[CLS]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[P